# Code With Me - Simpy - 00 - Introduction

This series of notebooks will lead you into Simpy step by step. 
We will build together some simulations in order to learn Simpy, understand why it works 
and learn some Python techniques required to get into Simpy. 

### Goal
This is a kind of tutorial. We will learn by doing.

I found the Simpy's documentation useful. However I thrived a lot in order to use it in an OOP (Object Oriented Programming). This series is a lot about using Simpy and OOP. It will not require a full knowledge of Python and OOP through. Advanced concepts will be explained.

### Prerequisites
- basic knowledge of Python
- basic knowledge of Object programming (concepts of class, method, attribute)

## What is Simpy ?

Simpy defines itself as stated below

> SimPy is a process-based discrete-event simulation framework based on standard Python - Syympy documentation.
    
First a simulation is a model or tool which goal is to behave like the real world in order to study some real world aspect.
For instance:
- people in a store and line length
- interaction between objects 

This is really useful when it is not possible to observe the real world.

A discrete-event simulation is a technique where the simulation mimics the real world by mimicking each even occuring in the real world and observe the process to check what happen.
> Alernative options for modeling is to use formulas, equations to compute the result. There are not always available or may not meet the needs of the survey.

Let us say that I need to evaluate if a departement store is safe. It is located in a beautiful old-fashion buiding. Its floor cannot support more than 500 persons. People may come in to visit the building, stay for a while, some may look for a specific item and finally some may buy something and eventually they leave. There are a limited number of checkout desks and people may have to wait in line in order to pay. By observing other stores in the area we featured out the expected number of people coming each hour. 

I may ask the following questions:
- does the number of people always stay under 500 ?
- where are these people, do they shop or stay in line ?
- how many checkout are required ?

## Simpy documentation 
Simpy documentation starts here [Simpy Overview](https://simpy.readthedocs.io/en/latest/) 

The best place to start is [Simpy in 10 minutes](https://simpy.readthedocs.io/en/latest/simpy_intro/index.html) 
  
You may also want to check [SimPy basics](https://simpy.readthedocs.io/en/latest/topical_guides/simpy_basics.html)
 in the Topical Guide to understand how it works.
 
The [API Reference](https://simpy.readthedocs.io/en/latest/api_reference/index.html) is useful to check the capabilities of Simpy classes.

# An example simulation

Let us star with a very simple example. We want to make a simulation of a gas station. 
- The gas station has 2 pumps 
- Cars will stay in line until they can refill their tanks

### install the simpy package

In [1]:
!pip install simpy

### simple process example

Let s start with a simple example. Cars wait at a gas station

In [2]:
# Tell we will use simpy
import simpy

# This is the process for each car
# move to the pup, fill the tank, go away
# when a car has filled it tank, the next car will come 
def car(env):
    while True:
        print('driver moves to the pump at %d' % env.now)
        # it takes 1 step to move in front of the pump
        yield env.timeout(1)

        print('driver refill the tank at %d' % env.now)
        # it takes 3 steps (says minutes) to fill the tank
        yield env.timeout(3)
        
        print('driver goes away at %d' % env.now)
        # it takes 1 step to go back into the car and leave the gas station
        yield env.timeout(1)

# the environement is the frame of the simulation
# the class Environment contains the tools to work with processes
# this piece of code creates an object of type Environement
env = simpy.Environment()
# prepare the simulation
# register  the function car as a process
env.process(car(env))
# run the simulation step by step for 10 steps 
print(f'Simulation starts at {env.now}')
env.run(until=10)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
driver moves to the pump at 0
driver refill the tank at 1
driver goes away at 4
driver moves to the pump at 5
driver refill the tank at 6
driver goes away at 9
Simulation stops at 10


**comments**

In this simulation the car process occured twice.
- the first car moved in at 0, starts refilling at 1 and went away at 4
- the next car moved in at 5, starts refilling at 6 and went away at 9

Time passing and gaps are caused by the timeout expressions.

The car function implements a loop in while True. It is an infinite llop. It will yield the car actions and messages again and again until the simulation end.
> You may check this by yourseilf by changing the simulation duration (the value of the parameter until)

## simple process consuming resource 

The reason why simulation are done is often ti check of some resource or capacity is used.

Say the gas station has 2 pumps. 

We cannot use an infinite process as multiple drivers may refill in parallel. 

The change is that now the process does not loop anymore. Instead we will create multiple car. A car will hold the pump (a resource in Simpy workd) and the next car comes in when the previous car release the pump.

In [3]:
# Tell we will use simpy
import simpy

# This is the process for each car
# move to the pup, fill the tank, go away
# the car is passed an id when it is created
# it is used in the print statement
# the f" " string is a formatted string. {} encloses the value to insert
def car(env, id):
    print(f'driver {id} moves to the pump at {env.now}')
    # it takes 1 step to move in front of the pump
    yield env.timeout(1)

    print(f'driver {id} refill the tank at {env.now}')
    # it takes 3 steps (says minutes) to fill the tank
    yield env.timeout(3)

    print(f'driver {id} goes away at {env.now}')
    # it takes 1 step to go back into the car and leave the gas station
    yield env.timeout(1)

env = simpy.Environment()
# prepare the simulation
# creates 5 cars
for i in range(0,5):
    # register  the function car as a process
    # the id is the current count
    env.process(car(env, i))
# run the simulation step by step for 10 steps 
print(f'Simulation starts at {env.now}')
env.run(until=10)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
driver 0 moves to the pump at 0
driver 1 moves to the pump at 0
driver 2 moves to the pump at 0
driver 3 moves to the pump at 0
driver 4 moves to the pump at 0
driver 0 refill the tank at 1
driver 1 refill the tank at 1
driver 2 refill the tank at 1
driver 3 refill the tank at 1
driver 4 refill the tank at 1
driver 0 goes away at 4
driver 1 goes away at 4
driver 2 goes away at 4
driver 3 goes away at 4
driver 4 goes away at 4
Simulation stops at 10


**comments**

Up to now each car starts at 0 and leave at 4 because the constraint on pump is not taken into account.


Let us add the pumps as a resource

In [4]:
# Tell we will use simpy
import simpy

# This is the process for each car
# move to the pup, fill the tank, go away
# the refill now hold a resource
def car(env, id):
    # The with statement is used when something is created 
    # and it is required to do a specific operation 
    # when it is not used anymore
    # Here with will keeo track of the resource request and 
    # release it when the end of the with statemebt is reached
    with pumps.request() as req:
        print(f'driver {id} arrives at {env.now}')
        # request the resource
        yield req
        
        print(f'driver {id} moves to the pump at {env.now}')
        # it takes 1 step to move in front of the pump
        yield env.timeout(1)

        print(f'driver {id} refill the tank at {env.now}')
        # it takes 3 steps (says minutes) to fill the tank
        # hold during 3 stepsto refill when the resource is available
        yield env.timeout(3)

        print(f'driver {id} goes away at {env.now}')
        # it takes 1 step to go back into the car and leave the gas station
        yield env.timeout(1)

        # the release is done automatically by the resource statement with

env = simpy.Environment()
# prepare the simulation
# creates 5 cars
for i in range(0,5):
    # register  the function car as a process
    # the id is the current count
    env.process(car(env, i))
# Creates a resource limited to 2 instance
pumps = simpy.Resource(env, capacity=2)
# run the simulation step by step for 10 steps 
print(f'Simulation starts at {env.now}')
env.run(until=10)
print(f'Simulation stops at {env.now}')

Simulation starts at 0
driver 0 arrives at 0
driver 1 arrives at 0
driver 2 arrives at 0
driver 3 arrives at 0
driver 4 arrives at 0
driver 0 moves to the pump at 0
driver 1 moves to the pump at 0
driver 0 refill the tank at 1
driver 1 refill the tank at 1
driver 0 goes away at 4
driver 1 goes away at 4
driver 2 moves to the pump at 5
driver 3 moves to the pump at 5
driver 2 refill the tank at 6
driver 3 refill the tank at 6
driver 2 goes away at 9
driver 3 goes away at 9
Simulation stops at 10


**comments**

Though they akk arrives at 0, cars 0 and 1 are served at 0, 
and cars 2 and 3 can access the pump at 5 
when cars 0 and 1 released the pump.

Car 4 never show up because the simulation ended before it could refill.
> You can check by yourself that it would have been served should the simulation be longer enough
Time passing and gaps are caused by the timeout expressions.


There are other types od resources. 
For instance, a Container might be used to track the quantity of gas available at the pump.